In [109]:

# coding: utf-8
import pandas as pd
# These lines set up the plotting funstionality and formatting.
import matplotlib 
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
matplotlib.use('Agg', warn = False)
#get_ipython().run_line_magic('matplotlib', 'inline')

import numpy as np


from sklearn.metrics import accuracy_score, precision_score, recall_score

from sklearn.model_selection import train_test_split

from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import itertools

import h2o
from h2o.automl import H2OAutoML

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [15]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,6 days 0 hours 54 mins
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.1
H2O cluster version age:,10 days
H2O cluster name:,H2O_from_python_xinyuzhang_ouafb2
H2O cluster total nodes:,1
H2O cluster free memory:,3.342 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


# 1. Load data

In [6]:
df = pd.read_pickle("processed_data.pickle")

# remove 3 object columns: p_grade, benefit, e_type
df = df.drop(['p_grade','benefit','e_type'], 1)


In [112]:
df.columns

Index(['Total Payments', 'Lump Sum Pay', 'Other Pay (Payroll Explorer)',
       'FMS Department', 'Job Class', 'Average Benefit Cost', 'Time', 'p_dep',
       'Rate', 'p_bonus', 't_bonus', 'o_pay', 'long', 'e_pay', 'insurance',
       'financial', 'Salary'],
      dtype='object')

# 2. Data spliting into test and train

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:16], df.iloc[:,16], test_size=0.33, random_state=42)


In [12]:
train = pd.concat([X_train, y_train], axis=1,join_axes=[X_train.index])
test = pd.concat([X_test, y_test], axis = 1, join_axes = [X_test.index])

# 3. use H2O to do automated model building and parameter training

In [16]:
h2oTest = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [17]:
h2oTrain = h2o.H2OFrame(train)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [18]:
aml=H2OAutoML(max_models=15, seed = 1)
aml.train(x = h2oTrain.columns, y = "Salary", training_frame=h2oTrain)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [19]:
lb = aml.leaderboard
lb.head(rows = lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20181105_211213,3.1793e+06,1783.06,3.1793e+06,725.01,nan
GBM_1_AutoML_20181105_211213,3.88976e+06,1972.25,3.88976e+06,791.208,nan
XGBoost_2_AutoML_20181105_211213,4.85811e+06,2204.11,4.85811e+06,918.586,nan
XGBoost_3_AutoML_20181105_211213,5.43043e+06,2330.33,5.43043e+06,1015.62,nan
XRT_1_AutoML_20181105_211213,8.90025e+06,2983.33,8.90025e+06,1162.5,nan
DRF_1_AutoML_20181105_211213,9.0217e+06,3003.62,9.0217e+06,1162.99,nan
StackedEnsemble_AllModels_AutoML_20181105_211213,4.10409e+08,20258.6,4.10409e+08,17194.1,nan
StackedEnsemble_BestOfFamily_AutoML_20181105_211213,1.71836e+09,41453.2,1.71836e+09,35318.2,nan
GLM_grid_1_AutoML_20181105_211213_model_1,1.71851e+09,41454.9,1.71851e+09,35319.2,nan


In [20]:
aml.leader

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_20181105_211213


ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 686012.2652432239
RMSE: 828.2585739026333
MAE: 563.1957504370328
RMSLE: NaN
Mean Residual Deviance: 686012.2652432239

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 2433921.4813856743
RMSE: 1560.1030355029998
MAE: 753.3167046843388
RMSLE: NaN
Mean Residual Deviance: 2433921.4813856743

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 3179303.4861155692
RMSE: 1783.060146522144
MAE: 725.0104331123545
RMSLE: NaN
Mean Residual Deviance: 3179303.4861155692
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,725.0117,32.675816,712.018,674.969,693.80396,736.1543,808.11346
mean_residual_deviance,3179327.0,816587.1,3063440.8,2162767.5,2292803.5,3006170.5,5371453.0
mse,3179327.0,816587.1,3063440.8,2162767.5,2292803.5,3006170.5,5371453.0
r2,0.9981547,0.0004622,0.9982519,0.9987214,0.9986555,0.9982344,0.9969103
residual_deviance,3179327.0,816587.1,3063440.8,2162767.5,2292803.5,3006170.5,5371453.0
rmse,1757.3151,213.50754,1750.2688,1470.635,1514.2006,1733.8312,2317.6394
rmsle,0.0,NaN,NaN,NaN,NaN,NaN,NaN


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2018-11-05 21:36:42,22 min 11.803 sec,0.0,72464.0991121,59436.5613438,5251045660.1293154,72349.2184601,59352.0224064,5234409411.7811594
,2018-11-05 21:36:42,22 min 12.030 sec,5.0,56554.3757333,46094.1630123,3198397414.5884237,56452.5895897,46042.6007912,3186894871.3840971
,2018-11-05 21:36:42,22 min 12.186 sec,10.0,44505.2948626,35921.1067508,1980721270.8029368,44399.3464881,35888.0947334,1971301968.5707896
,2018-11-05 21:36:43,22 min 12.354 sec,15.0,35186.6292065,28201.9408835,1238098874.9159818,35071.7362866,28176.5120574,1230026686.1558406
,2018-11-05 21:36:43,22 min 12.521 sec,20.0,27768.0008490,22066.4224011,771061871.1477618,27662.4544458,22045.9857803,765211385.9686179
---,---,---,---,---,---,---,---,---,---
,2018-11-05 21:39:24,24 min 53.534 sec,1155.0,834.9169298,567.7301326,697086.2796984,1563.4995583,756.9622240,2444530.8687294
,2018-11-05 21:39:25,24 min 55.188 sec,1160.0,833.4653585,566.4855722,694664.5038818,1562.6905523,755.9120653,2442001.7620951
,2018-11-05 21:39:27,24 min 56.847 sec,1165.0,831.3410824,564.9154586,691127.9952386,1561.8799571,754.5913582,2439469.0003443
,2018-11-05 21:39:29,24 min 58.518 sec,1170.0,829.1323071,563.6323581,687460.3827516,1560.8289232,753.6601954,2436186.9274145



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
TotalPayments,6168.0,1.0,0.1928645
Rate,4983.0,0.8078794,0.1558113
o_pay,3670.0,0.5950065,0.1147556
OtherPay(PayrollExplorer),3289.0,0.5332361,0.1028423
JobClass,2414.0,0.3913748,0.0754823
e_pay,2299.0,0.3727302,0.0718864
LumpSumPay,1579.0,0.2559987,0.0493731
p_bonus,1536.0,0.2490272,0.0480285
AverageBenefitCost,1457.0,0.2362192,0.0455583
Time,1344.0,0.2178988,0.0420250


# 4. Save the Xgboost Model: 

In [127]:
#save the model:
model_path = h2o.save_model(model = aml.leader, path = "./",force = True)

In [128]:
print(model_path)

/Users/xinyuzhang/Desktop/hackthon_gwu/XGBoost_1_AutoML_20181105_211213


In [129]:
type(model_path)

str

# 5. Prediction and output

In [25]:
preds = aml.leader.predict(h2oTest)
preds = preds.as_data_frame()

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [32]:
train_preds = aml.leader.predict(h2oTrain)
train_preds = train_preds.as_data_frame()

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [26]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, preds)

3007148.555364304

In [27]:
print(aml.leader)

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_20181105_211213


ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 686012.2652432239
RMSE: 828.2585739026333
MAE: 563.1957504370328
RMSLE: NaN
Mean Residual Deviance: 686012.2652432239

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 2433921.4813856743
RMSE: 1560.1030355029998
MAE: 753.3167046843388
RMSLE: NaN
Mean Residual Deviance: 2433921.4813856743

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 3179303.4861155692
RMSE: 1783.060146522144
MAE: 725.0104331123545
RMSLE: NaN
Mean Residual Deviance: 3179303.4861155692
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,725.0117,32.675816,712.018,674.969,693.80396,736.1543,808.11346
mean_residual_deviance,3179327.0,816587.1,3063440.8,2162767.5,2292803.5,3006170.5,5371453.0
mse,3179327.0,816587.1,3063440.8,2162767.5,2292803.5,3006170.5,5371453.0
r2,0.9981547,0.0004622,0.9982519,0.9987214,0.9986555,0.9982344,0.9969103
residual_deviance,3179327.0,816587.1,3063440.8,2162767.5,2292803.5,3006170.5,5371453.0
rmse,1757.3151,213.50754,1750.2688,1470.635,1514.2006,1733.8312,2317.6394
rmsle,0.0,NaN,NaN,NaN,NaN,NaN,NaN


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2018-11-05 21:36:42,22 min 11.803 sec,0.0,72464.0991121,59436.5613438,5251045660.1293154,72349.2184601,59352.0224064,5234409411.7811594
,2018-11-05 21:36:42,22 min 12.030 sec,5.0,56554.3757333,46094.1630123,3198397414.5884237,56452.5895897,46042.6007912,3186894871.3840971
,2018-11-05 21:36:42,22 min 12.186 sec,10.0,44505.2948626,35921.1067508,1980721270.8029368,44399.3464881,35888.0947334,1971301968.5707896
,2018-11-05 21:36:43,22 min 12.354 sec,15.0,35186.6292065,28201.9408835,1238098874.9159818,35071.7362866,28176.5120574,1230026686.1558406
,2018-11-05 21:36:43,22 min 12.521 sec,20.0,27768.0008490,22066.4224011,771061871.1477618,27662.4544458,22045.9857803,765211385.9686179
---,---,---,---,---,---,---,---,---,---
,2018-11-05 21:39:24,24 min 53.534 sec,1155.0,834.9169298,567.7301326,697086.2796984,1563.4995583,756.9622240,2444530.8687294
,2018-11-05 21:39:25,24 min 55.188 sec,1160.0,833.4653585,566.4855722,694664.5038818,1562.6905523,755.9120653,2442001.7620951
,2018-11-05 21:39:27,24 min 56.847 sec,1165.0,831.3410824,564.9154586,691127.9952386,1561.8799571,754.5913582,2439469.0003443
,2018-11-05 21:39:29,24 min 58.518 sec,1170.0,829.1323071,563.6323581,687460.3827516,1560.8289232,753.6601954,2436186.9274145



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
TotalPayments,6168.0,1.0,0.1928645
Rate,4983.0,0.8078794,0.1558113
o_pay,3670.0,0.5950065,0.1147556
OtherPay(PayrollExplorer),3289.0,0.5332361,0.1028423
JobClass,2414.0,0.3913748,0.0754823
e_pay,2299.0,0.3727302,0.0718864
LumpSumPay,1579.0,0.2559987,0.0493731
p_bonus,1536.0,0.2490272,0.0480285
AverageBenefitCost,1457.0,0.2362192,0.0455583
Time,1344.0,0.2178988,0.0420250


In [28]:
aml.leader.model_performance(h2oTest)


ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 3007148.4642519224
RMSE: 1734.1131636233902
MAE: 769.6179810823162
RMSLE: NaN
Mean Residual Deviance: 3007148.4642519224


In [30]:
#save test_pred to pickle file
preds.to_pickle("y_pred.pickle")

In [34]:
# save train_pred to pickle file
train_preds.to_pickle("train_y_pred.pickle")

In [65]:
#transform y_train and combine y_train_pred as a dataframe
dd = y_train.to_frame().reset_index(drop=True)
aa = pd.DataFrame()
aa["y_train"] = dd
aa["y_train_pred"] = train_preds

In [ ]:
# substraction two columns to compute residuals:
aa["sub"] = aa[["y_train","y_train_pred"]].sum(axis=1)
yy = aa[["y_train","sub"]].sub(aa["y_train_pred"],axis=0)
# save residuals 
yy.to_pickle("graph_train.pickle")

In [113]:
#transform y_test and combine y_test_pred as a dataframe
tt = y_test.to_frame().reset_index(drop=True)
bb = pd.DataFrame()
bb["y_test"] = tt
bb["y_test_pred"] = preds

In [117]:
# substraction two columns to compute residuals:
bb["sub"] = bb[["y_test","y_test_pred"]].sum(axis=1)
yy2 = bb[["y_test_pred","sub"]].sub(bb["y_test"],axis=0)

In [121]:
bb.head()

,y_test,y_test_pred,sub
0,13307.81,13361.247070,26669.057070
1,2170.40,1795.662598,3966.062598
2,52976.04,54344.175781,107320.215781
3,25515.20,26044.552734,51559.752734
4,98724.80,99007.437500,197732.237500


In [126]:
yy2.to_pickle("graph_test.pickle")

In [125]:
yy2.head()

,y_test_pred,sub
0,53.437070,13361.247070
1,-374.737402,1795.662598
2,1368.135781,54344.175781
3,529.352734,26044.552734
4,282.637500,99007.437500
